In [9]:
import matplotlib.pyplot as plt 
import pandas as pd
import math
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU 

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import psycopg2
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt 
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'white'
import seaborn as sns

import keras_functions
from keras_functions import *

In [2]:
con = psycopg2.connect(database='codeforces', user='Joy')
cur = con.cursor()

In [3]:
binvars = get_categorical_variables([
    'all_participanttypes',
    'all_tags',
    'all_language'
])

# Initial pipeline

In [4]:
with open('set_train.txt') as f:
    train_handles = [h.strip() for h in f.readlines()]
with open('set_test.txt') as f:
    test_handles = [h.strip() for h in f.readlines()]

In [5]:
user_batch = 10

In [75]:
reload(keras_functions)
dflist = []
for i in range(user_batch):
    filename = 'rnn_train/' + train_handles[i] + ".csv"
    print filename
    dflist.append(keras_functions.get_user_data(train_handles[i], binvars=binvars, month=3, maxtimepts=500))

rnn_train/2difficult4me.csv
rnn_train/priyanka_1998.csv
rnn_train/vatr.csv
rnn_train/shantanushanto.csv
rnn_train/KMPArthas.csv
rnn_train/914295860.csv
rnn_train/diver_ru.csv
rnn_train/yangzx1004.csv
rnn_train/BigChampion.csv
rnn_train/lbn187.csv


In [ ]:
user='yj12'
month=3
maxtimepts = 500
# set some parameters for normalization
max_change = 200.0

# -----------------------------
# Load data
data = pd.read_csv('rnn_train/%s.csv'%user)

# drop the first contest--we don't have a "real" change from a null expectation here
data.sort_values('ratingupdatetimeseconds', inplace=True)
firstcid = data.contestid.values[0]
data.drop(data.index[data.contestid == firstcid], axis=0, inplace=True)

# -----------------------------
# binarize some variables
data[binvars] = data[binvars].fillna(value=0)

for b in binvars:
    data.loc[ data[b] > 0, b] = 1

# -----------------------------
# remove information for other months
df_data = data
for m in range(1,6):
    if m == month:
        continue
    name1 = "delta_smoothed_%dmonths" % m
    name2 = "smoothed_%dmonths" % m

    df_data.drop([name1, name2], axis=1, inplace=True)

In [ ]:
df_train = df_data.drop(['handle', 'index'], axis=1)
df_train.fillna(value=0, inplace=True)

# -----------------------------
# Feature scaling
y_column = 'delta_smoothed_%dmonths' % month
colnames = list(df_train.columns.values)

cids = df_train.contestid

scaler = MinMaxScaler(feature_range=(0, 1))
df_train_scaled = scaler.fit_transform(df_train)
df_train_scaled = pd.DataFrame(df_train_scaled)
df_train_scaled.columns = colnames

# add back in colnames that should not be scaled
df_train_scaled['contestid'] = cids
df_train_scaled[y_column] = df_train[y_column]

# scale the y column wrt to max change possible
df_train_scaled[y_column] /= max_change

# -----------------------------
# Group by contest
groups = df_train_scaled.groupby('contestid')

# -----------------------------
# create list of inputs for training
trainlist = []
ylist = []

In [ ]:
for k, v in groups:
    v.is_copy = False

    v.drop('contestid', axis=1, inplace=True)
    y = v.loc[:, y_column].values[0]
    v.drop(y_column, inplace=True, axis=1)

    trainlist.append(v)
    ylist.append(y)

ary = np.array(ylist)
print len(groups)
print trainlist

